### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [17]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [18]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [19]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [20]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [21]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [22]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [23]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [24]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [25]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [26]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [27]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [28]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [29]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [30]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [31]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [32]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [33]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [34]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [35]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno y analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámeteros
de instanciación del vectorizador y los modelos (probar modelos de Naïve Bayes Multinomial
y ComplementNB).

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras, tomando 5 y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Resolviendo:

Inciso 1:

In [36]:
# Importa librerias extra a las ya convocadas
import random

In [37]:
# Selecciona 5 documentos al azar de los datos de entrenamiento
random.seed(42)  # Fija la seed para hacer que los resultados reproducibles
indices_aleatorios = random.sample(range(X_train.shape[0]), 5)

In [38]:
# Extrae los 5 documentos seleccionados
documentos_seleccionados = X_train[indices_aleatorios]

In [39]:
# Calcula la similitud coseno entre los documentos seleccionados y todos los documentos
similaridades = cosine_similarity(documentos_seleccionados, X_train)

In [40]:
# Muestra la clase de los documentos seleccionados y de sus 5 documentos mas similares
for i, idx in enumerate(indices_aleatorios):
    # Obtenemos las 5 posiciones más similares (excluyendo el propio documento)
    similares_idx = np.argsort(similaridades[i])[::-1][1:6]
    
    # Nombre de la clase del documento seleccionado
    clase_seleccionada = newsgroups_train.target_names[y_train[idx]]
    
    print(f"Documento {i+1} (Índice {idx}): Clase {clase_seleccionada}")
    
    print("Documentos más similares:")
    for j, similar_idx in enumerate(similares_idx):
        # Nombre de la clase del documento similar
        clase_similar = newsgroups_train.target_names[y_train[similar_idx]]
        print(f"  - Documento {j+1}: Índice {similar_idx}, Clase {clase_similar}, Similitud: {similaridades[i][similar_idx]:.4f}")
    
    print("\n")

Documento 1 (Índice 10476): Clase rec.sport.hockey
Documentos más similares:
  - Documento 1: Índice 5064, Clase rec.sport.hockey, Similitud: 0.2250
  - Documento 2: Índice 9623, Clase talk.politics.mideast, Similitud: 0.2174
  - Documento 3: Índice 10575, Clase sci.crypt, Similitud: 0.2164
  - Documento 4: Índice 10836, Clase alt.atheism, Similitud: 0.2126
  - Documento 5: Índice 2350, Clase sci.crypt, Similitud: 0.2111


Documento 2 (Índice 1824): Clase comp.sys.mac.hardware
Documentos más similares:
  - Documento 1: Índice 9921, Clase comp.sys.mac.hardware, Similitud: 0.3542
  - Documento 2: Índice 6364, Clase comp.sys.mac.hardware, Similitud: 0.3132
  - Documento 3: Índice 5509, Clase comp.sys.mac.hardware, Similitud: 0.3041
  - Documento 4: Índice 2641, Clase comp.sys.mac.hardware, Similitud: 0.2504
  - Documento 5: Índice 4359, Clase comp.sys.mac.hardware, Similitud: 0.2417


Documento 3 (Índice 409): Clase comp.graphics
Documentos más similares:
  - Documento 1: Índice 3444, Cla

In [41]:
# Muestra el contenido de los documentos seleccionados y de sus 5 documentos mas similares
for i, idx in enumerate(indices_aleatorios):
    # Obtenemos las 5 posiciones más similares (excluyendo el propio documento)
    similares_idx = np.argsort(similaridades[i])[::-1][1:6]
    
    # Nombre de la clase del documento seleccionado
    clase_seleccionada = newsgroups_train.target_names[y_train[idx]]
    
    print(f"Documento {i+1} (Índice {idx}): Clase {clase_seleccionada}")
    print(f"Contenido del documento:\n{newsgroups_train.data[idx][:500]}...")  # Mostrar solo los primeros 500 caracteres
    print("\nDocumentos más similares:")
    
    for j, similar_idx in enumerate(similares_idx):
        # Nombre de la clase del documento similar
        clase_similar = newsgroups_train.target_names[y_train[similar_idx]]
        
        # Mostrar contenido de los documentos similares
        print(f"  - Documento {j+1}: Índice {similar_idx}, Clase {clase_similar}, Similitud: {similaridades[i][similar_idx]:.4f}")
        print(f"    Contenido del documento:\n{newsgroups_train.data[similar_idx][:500]}...")  # Mostrar solo los primeros 500 caracteres
        print("\n")
    
    print("="*80)  # Separador para cada grupo de documentos

Documento 1 (Índice 10476): Clase rec.sport.hockey
Contenido del documento:
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are televised nationally or even
locally in the US and how much precedence they take over, say, local new...

Documentos más similares:
  - Documento 1: Índice 5064, Clase rec.sport.hockey, Similitud: 0.2250
    Contenido del documento:

I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are THE most exciting
games to watch....


  - Documento 2: Índice 9623, Clase talk.politics.mi

Observaciones:

En cuanto al documento 1, solo el documento con mayor similitud pertenece a la misma clase (hockey). El resto presenta temas diversos como política y oriente medio. Solo el 4to presenta cierta similitud dado que el tema es atletismo. El caso de mayor similitud tiene sentido dado que hablan de una competencia de "play-off".

En cuanto al documento 2, en todos los casos la similitud se da en documentos con la misma etiqueta: hardware. No obstante, llama la atención que los valores de similitud inician desde 0.3542 hacia abajo. Es decir que se tratan de valores más cercanos a 0 que a 1. En la mayoría de documentos se mencionan modelos de productos de hardware (usados como nombre propio), siendo posiblemente un factor del buen desempeño de similaridad.

El análisis de lo que sucede con el documento 3 es similar al anterior. Se logró encontrar documentos que pertenezcan a la misma clase: gráficos de computadora. Nuevamente, se aprecian términos específicos de modelos de productos del rubro que permiten (posiblemente) a que tuvieran peso en el cálculo de la similaridad.

Pasando a la documento 4 que trata de autos, los últimos 3 del ranking de similitud pertenecen a la misma categoría: autos. Esto es curioso dado que si bien el que representó mayor similitud es orientado a motocicletas, luego sigue un documento sobre hardware. En este último caso, parece ser porque el tema a tratar es sobre el sonido del producto, un caso del auto y del otro una computadora.

Por último, en el documento 5 de clase hockey, fue bastante correspondido con los demás documentos. Considerando el ranking descendente de similitud, los documentos 2 a 4 pertenecen también a hockey dejando al 5to perteneciendo a básquet (que tiene afinidad por ser deporte). No obstante, el de mayor similitud es sobre religión cristiana. Es posible que la similitud se haya dado por que en ambos casos se habla sobre un tal Peter (apóstol y jugador respectivamente).

Inciso 2:

In [43]:
# Parametros para el vectorizador
tfidf_params = {
    'ngram_range': [(1, 1), (1, 2)],  # Unigrams y bigrams
    'max_features': [1000, 5000, None],  # Prueba diferentes numeros de caracteristicas
    'stop_words': [None, 'english']  # Sin palabras vacias y con palabras vacias en ingles
}

In [44]:
# Parametros para los modelos de Naïve Bayes
nb_params = {
    'alpha': [1.0, 0.5, 0.1],  # Diferentes valores para el suavizado
}

In [45]:
# Crea el vectorizador
best_score = 0
best_model = None
best_vectorizer = None

Se realiza un primer experimento utilizando TfidfVectorizer:

In [46]:
# Bucle para probar diferentes configuraciones de vectorizacion con Tfidf
for ngram in tfidf_params['ngram_range']:
    for max_feat in tfidf_params['max_features']:
        for stop_words in tfidf_params['stop_words']:
            # Instancia y ajusta el vectorizador
            tfidfvect = TfidfVectorizer(ngram_range=ngram, max_features=max_feat, stop_words=stop_words)
            X_train = tfidfvect.fit_transform(newsgroups_train.data)
            X_test = tfidfvect.transform(newsgroups_test.data)
            
            # Prueba ambos modelos de Naïve Bayes
            for model_name, model_class in zip(['MultinomialNB', 'ComplementNB'], [MultinomialNB, ComplementNB]):
                clf = model_class()
                
                # Ajusta el modelo
                clf.fit(X_train, newsgroups_train.target)
                y_pred = clf.predict(X_test)
                
                # Calcula el F1-score macro
                f1 = f1_score(newsgroups_test.target, y_pred, average='macro')
                
                print(f'Modelo: {model_name}, ngram_range: {ngram}, max_features: {max_feat}, stop_words: {stop_words}, F1-score macro: {f1:.4f}')
                
                # Guarda el mejor modelo y sus parametros
                if f1 > best_score:
                    best_score = f1
                    best_model = model_name
                    best_vectorizer = (ngram, max_feat, stop_words)

print(f'\nMejor modelo: {best_model} con F1-score macro de {best_score:.4f} usando ngram_range {best_vectorizer[0]}, max_features {best_vectorizer[1]}, stop_words {best_vectorizer[2]}')

Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 1000, stop_words: None, F1-score macro: 0.4783
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 1000, stop_words: None, F1-score macro: 0.4664
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 1000, stop_words: english, F1-score macro: 0.5037
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 1000, stop_words: english, F1-score macro: 0.4835
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 5000, stop_words: None, F1-score macro: 0.6033
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 5000, stop_words: None, F1-score macro: 0.6344
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 5000, stop_words: english, F1-score macro: 0.6251
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 5000, stop_words: english, F1-score macro: 0.6351
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: None, stop_words: None, F1-score macro: 0.5854
Modelo: ComplementNB, ngram_range: (1, 1), max_

In [47]:
# Crea el vectorizador
best_score = 0
best_model = None
best_vectorizer = None

Se realiza un segundo experimento utilizando CountVectorizer:

In [48]:
# Bucle para probar diferentes configuraciones de vectorizacion con Count
for ngram in tfidf_params['ngram_range']:
    for max_feat in tfidf_params['max_features']:
        for stop_words in tfidf_params['stop_words']:
            # Instancia y ajusta el vectorizador
            countvect = CountVectorizer(ngram_range=ngram, max_features=max_feat, stop_words=stop_words)
            X_train = countvect.fit_transform(newsgroups_train.data)
            X_test = countvect.transform(newsgroups_test.data)
            
            # Prueba ambos modelos de Naïve Bayes
            for model_name, model_class in zip(['MultinomialNB', 'ComplementNB'], [MultinomialNB, ComplementNB]):
                clf = model_class()
                
                # Ajusta el modelo
                clf.fit(X_train, newsgroups_train.target)
                y_pred = clf.predict(X_test)
                
                # Calcula el F1-score macro
                f1 = f1_score(newsgroups_test.target, y_pred, average='macro')
                
                print(f'Modelo: {model_name}, ngram_range: {ngram}, max_features: {max_feat}, stop_words: {stop_words}, F1-score macro: {f1:.4f}')
                
                # Guarda el mejor modelo y sus parametros
                if f1 > best_score:
                    best_score = f1
                    best_model = model_name
                    best_vectorizer = (ngram, max_feat, stop_words)

print(f'\nMejor modelo: {best_model} con F1-score macro de {best_score:.4f} usando ngram_range {best_vectorizer[0]}, max_features {best_vectorizer[1]}, stop_words {best_vectorizer[2]}')

Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 1000, stop_words: None, F1-score macro: 0.4178
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 1000, stop_words: None, F1-score macro: 0.3778
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 1000, stop_words: english, F1-score macro: 0.4454
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 1000, stop_words: english, F1-score macro: 0.4030
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 5000, stop_words: None, F1-score macro: 0.5561
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 5000, stop_words: None, F1-score macro: 0.5540
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: 5000, stop_words: english, F1-score macro: 0.5709
Modelo: ComplementNB, ngram_range: (1, 1), max_features: 5000, stop_words: english, F1-score macro: 0.5644
Modelo: MultinomialNB, ngram_range: (1, 1), max_features: None, stop_words: None, F1-score macro: 0.5121
Modelo: ComplementNB, ngram_range: (1, 1), max_

Observaciones:

Se realizaron dos experiemntos en paralelo, operando en ambos de la misma forma pero con diferentes técnicas de vectorización: Por *Term Frequency-Inverse Document Frequency* y *Conteo*.
En ambos se uilizaron dos modelos de Bayes distintos y diferentes parámetros. En cuanto a *n_gram_rane*, se consideró que los elementos del vocabulario tuvieran 1 ó 2 palabras. Además se probaron dos límites (y que no haya ninguno) a la cantidad de características, utilizando *max_features*. También, se consideró que se pruebe con una lista predefinida de palabras en inglés que no fueran incluidas y que si lo fueran, haciendo uso de *stop_words*. Estas variaciones se tomaron como grilla de hiperparámetros. También se consideraron dos valores no nulos (y el valor nulo) de *alpha* para expermientar con el problema de "probabilidad nula".

Considerando lo anterior, se probó con dos modelos: *Multinomial Naïve Bayes* y *Complemento de Naïve Bayes*. Se probabron todas las variaciones con ambos modelos y se pudo ver que en general utilizando la vectorización *TF-IDF* arrojó valores de f1-score mejor que con el método por conteo. No obstante, en ambos casos el modelo *Complemento de Naïve Bayes* fue el que mejor desempeño mostró con esta métrica, utilizando en ambas vectorizaciones ngram_range: (1, 2), max_features: None, stop_words: english como hiperparámetros.

Inciso 3:

In [49]:
# Define el vectorizador
tfidfvect = TfidfVectorizer()

In [50]:
# Define la matriz documento-termino
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [51]:
# Transpone la matriz documento-termino para obtener la matriz termino-documento
X_train_transposed = X_train.T

In [52]:
# Obtiene el vocabulario (lista de palabras)
vocabulario = tfidfvect.get_feature_names_out()

# Muestra todas las palabras
for i in range(len(vocabulario)):
    print(vocabulario[i])


00
000
0000
00000
000000
00000000
0000000004
00000000b
00000001
00000001b
00000010
00000010b
00000011
00000011b
00000074
00000093
000000e5
00000100
00000100b
00000101
00000101b
00000110
00000110b
00000111
00000111b
000005102000
00000510200001
00000ee5
00001000
00001000b
00001001
00001001b
00001010
00001010b
00001011
00001011b
000010af
00001100
00001100b
00001101
00001101b
00001110
00001110b
00001111
00001111b
000042
000062david42
000094
0001
00010000
00010000b
00010001
00010001b
00010010
00010010b
00010011
00010011b
000100255pixel
00010100
00010100b
00010101
00010101b
00010110
00010110b
00010111
00010111b
00011000
00011000b
00011001
00011001b
00011010
00011010b
00011011
00011011b
00011100
00011100b
00011101
00011101b
00011110
00011110b
00011111
00011111b
00014
000152
0001mpc
0001x7c
0002
0003
0004
00041032
0004136
0004246
0004422
00044513
0004847546
0005
0005895485
0007
00072
0007259d
00072741
00072840
00072a27
000773b9
00077bbe
00090711
000ds
000k
000mi
000miles
000th
000usd
001
0010


In [11]:
# Toma 5 palabras elegidas manualmente (a modo visual, no se usan de esta forma)
word_1 = vocabulario[28177]  # cobra
word_2 = vocabulario[43346]  # ghost
word_3 = vocabulario[63337]  # move
word_4 = vocabulario[78362]  # rider
word_5 = vocabulario[96591]  # wich

In [53]:
# Crea una lista con los indices de las palabras seleccionadas
selected_word_indices = [28177, 43346, 63337, 78362, 96591]

In [54]:
# Extrae las filas correspondientes a estas palabras de la matriz termino-documento
selected_words_vectors = X_train_transposed[selected_word_indices]

In [55]:
# Calcula la similitud coseno entre las palabras seleccionadas y todas las palabras del vocabulario
cosine_similarities = cosine_similarity(selected_words_vectors, X_train_transposed)

In [56]:
# Muestrar las 5 palabras mas similares para cada palabra seleccionada
for i, word_idx in enumerate(selected_word_indices):
    # Obtener los índices de las 5 palabras más similares (excluyendo la palabra en sí misma)
    most_similar_indices = np.argsort(-cosine_similarities[i])[1:6]
    
    # Mostrar las 5 palabras más similares
    print(f"Palabras más similares a '{vocabulario[word_idx]}':")
    for idx in most_similar_indices:
        print(f"  {vocabulario[idx]} (similaridad: {cosine_similarities[i][idx]:.4f})")
    print("\n")


Palabras más similares a 'cobra':
  padlock (similaridad: 0.5770)
  jele (similaridad: 0.3800)
  tsoft (similaridad: 0.3800)
  thetech (similaridad: 0.3800)
  mirage (similaridad: 0.3525)


Palabras más similares a 'ghost':
  suptermarket (similaridad: 0.5594)
  haunts (similaridad: 0.5579)
  tore (similaridad: 0.4986)
  werewolf (similaridad: 0.4724)
  superman (similaridad: 0.4427)


Palabras más similares a 'move':
  sneak (similaridad: 0.3197)
  nationalities (similaridad: 0.2204)
  smart (similaridad: 0.2144)
  electron (similaridad: 0.1926)
  rfk (similaridad: 0.1822)


Palabras más similares a 'rider':
  pigeonhole (similaridad: 0.3186)
  swerve (similaridad: 0.3080)
  mcqueen (similaridad: 0.2831)
  thumper (similaridad: 0.2755)
  manouvres (similaridad: 0.2707)


Palabras más similares a 'wich':
  pcmith (similaridad: 0.8032)
  thurbin (similaridad: 0.8032)
  jd3824r00 (similaridad: 0.8032)
  rks (similaridad: 0.8032)
  zwl (similaridad: 0.8032)




Observaciones:

Lo primero que llama la atención es que en todos los casos las 5 palabras con mayor similaridad son en realidad bastante diferentes a la palabra original. No obstante, esto toma sentido si se tiene presente como está armada de la matriz término-documento.

Por un lado, puede deberse a que las palabras comparten un contexto. Sabemos que la similitud coseno mide la orientación de los vectores en el espacio vectorial, no su distancia en sí. Si por ejemplo "cobra", "padlock", "jele", "tsoft", etc, aparecen en contextos similares en los documentos, su representación vectorial puede estar cerca, lo que resulta en una alta similitud. Es decir que pueden estar relacionadas temáticamente en el corpus, aunque no tengan un significado semántico directo.

Además, es posible que estas palabras aparezcan en documentos similares, lo que puede indicar que se discuten en temas relacionados. Por ejemplo, si un documento trata sobre automovilismo y menciona "rider" y "manouvres", apareciendo ambos en el mismo documento o similares, crea un vínculo entre estas palabras.

También podría suceder que las palabras que aparecen en muchos documentos juntos tienden a ser representadas por vectores similares.